<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/iris_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd
import torch
from torch import nn

import numpy as np

In [78]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [79]:
from sklearn.model_selection import train_test_split

In [80]:
from sklearn import datasets
import pandas as pd

# Load the Iris dataset
iris = datasets.load_iris()

# Create a DataFrame from the dataset
iris_df = pd.DataFrame(iris.data)
iris_df['class'] = iris.target
iris_df.columns = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']

In [81]:
df = iris_df.sample(frac = 1)
df_train,df_test = train_test_split(df,test_size = 0.1, random_state = 1)
y_train = df_train['class']
y_test = df_test['class']
X_train = df_train.drop('class', axis = 1)
X_test = df_test.drop('class', axis = 1)

In [82]:
y_train_t,y_test_t,X_train_t,X_test_t = torch.from_numpy(y_train.to_numpy()),torch.from_numpy(y_test.to_numpy()),torch.from_numpy(X_train.to_numpy()),torch.from_numpy(X_test.to_numpy())

In [83]:
y_train_t = y_train_t.type(torch.long).to(device)
y_test_t = y_test_t.type(torch.long).to(device)
X_train_t = X_train_t.type(torch.float32).to(device)
X_test_t = X_test_t.type(torch.float32).to(device)

In [84]:
class nn_model(nn.Module):
  def __init__(self):
    super().__init__()
    self.stack = nn.Sequential(
        nn.Linear(4,4),
        nn.ReLU(),
        nn.Linear(4,3)
    )

  def forward(self, X ):
    return self.stack(X)

In [85]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [86]:
model_0 = nn_model()

In [87]:
model_0.to(device)

nn_model(
  (stack): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=3, bias=True)
  )
)

In [88]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.01)

In [89]:
torch.manual_seed(42)
epochs = 100


In [90]:
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train_t) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train_t)
    acc = accuracy_fn(y_true=y_train_t,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_0(X_test_t)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test_t)
      test_acc = accuracy_fn(y_true=y_test_t,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 1.62290, Acc: 65.19% | Test Loss: 1.09657, Test Acc: 80.00%
Epoch: 10 | Loss: 0.93167, Acc: 65.19% | Test Loss: 0.81527, Test Acc: 80.00%
Epoch: 20 | Loss: 0.77631, Acc: 93.33% | Test Loss: 0.75246, Test Acc: 100.00%
Epoch: 30 | Loss: 0.66363, Acc: 88.15% | Test Loss: 0.62125, Test Acc: 100.00%
Epoch: 40 | Loss: 0.56217, Acc: 91.11% | Test Loss: 0.50608, Test Acc: 100.00%
Epoch: 50 | Loss: 0.48508, Acc: 93.33% | Test Loss: 0.42707, Test Acc: 100.00%
Epoch: 60 | Loss: 0.42618, Acc: 96.30% | Test Loss: 0.37933, Test Acc: 100.00%
Epoch: 70 | Loss: 0.37866, Acc: 97.04% | Test Loss: 0.33866, Test Acc: 100.00%
Epoch: 80 | Loss: 0.33890, Acc: 97.04% | Test Loss: 0.30711, Test Acc: 100.00%
Epoch: 90 | Loss: 0.30432, Acc: 97.78% | Test Loss: 0.27939, Test Acc: 100.00%
